In [1]:
import pandas as pd
import numpy as np
from torch_geometric.data import Data
from sklearn.preprocessing import LabelEncoder
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
df_orignal = pd.read_csv("all_data.csv")

df1=df_orignal

In [3]:
for i in range(df1["Month"].min()+1,df1["Month"].max()):
    df = df1.loc[df1["Month"]==i]

    graph = nx.from_pandas_edgelist(df, source='From_Account_id', target='To_Account_id',create_using=nx.DiGraph())
    in_degree_centrality = nx.in_degree_centrality(graph)
    out_degree_centrality = nx.out_degree_centrality(graph)    
    centrality_df = pd.DataFrame({
        'in_degree_centrality': list(in_degree_centrality.values()),
        'out_degree_centrality': list(out_degree_centrality.values())
    },index=list(in_degree_centrality.keys()))


    outgoing = df.groupby('From_Account_id')['amount'].sum().reset_index()
    outgoing.rename(columns={'amount': 'Outgoing_Amount'}, inplace=True)
    incoming = df.groupby('To_Account_id')['amount'].sum().reset_index()
    incoming.rename(columns={'amount': 'Incoming_Amount'}, inplace=True)
    total_amounts = pd.merge(outgoing, incoming, left_on='From_Account_id', right_on='To_Account_id', 
                             how='outer', suffixes=('_source', '_target'))
    total_amounts.fillna(0, inplace=True)    
    total_amounts = total_amounts[['Outgoing_Amount', 'Incoming_Amount']]
    total_amounts.index=list(in_degree_centrality.keys())
    
    node_features = pd.concat([total_amounts, centrality_df], axis=1)
    node_features.to_csv(f"month-{i}-nodes.csv",index=True)

In [5]:
node_features

,Outgoing_Amount,Incoming_Amount,in_degree_centrality,out_degree_centrality
1,45960.0,0.0,0.000000e+00,1.340697e-06
4734757,2180.0,0.0,4.692439e-06,0.000000e+00
4836760,1830.0,0.0,5.027613e-06,0.000000e+00
4837634,0.0,44000.0,4.022091e-06,0.000000e+00
4320939,320.0,0.0,3.351742e-06,0.000000e+00
...,...,...,...,...
4874009,1020.0,2000.0,0.000000e+00,3.351742e-07
4874014,0.0,1000.0,0.000000e+00,6.703484e-06
2547242,10000.0,0.0,3.351742e-07,0.000000e+00
4874018,0.0,900.0,0.000000e+00,2.011045e-06
